# SIMPLE NEURAL NETWORK TO CARTIGORIZE MUSIC
#### author: Rhema Ike

In [2]:
import pandas as pd
import tensorflow as tf

In [3]:
import os
import random

In [64]:
# Load Data

DATA_DIR = "DataSets"
file_name = "SongDataSetLess_4000.xlsx"

file_name = os.path.join(DATA_DIR, file_name)

df = pd.read_excel(file_name)
df.drop(["name","artist"], axis=1,inplace=True)


PLAYLIST_LEN = 1000
target = ([1]* PLAYLIST_LEN) + ([2]* PLAYLIST_LEN) + ([3]* PLAYLIST_LEN) + ([4]* PLAYLIST_LEN)
target_df = pd.DataFrame(target, columns=["target"])

df = pd.concat([df,target_df], axis=1)


In [60]:
df.iloc[2400]

name                   Believer
artist              Crazy Fluke
key                          11
liveness                   0.22
instrumentalness          0.874
loudness                 -7.407
mode                          0
speechiness              0.0813
tempo                   122.988
valence                   0.684
danceability              0.723
energy                    0.869
acousticness           0.000404
target                        3
Name: 2400, dtype: object

In [61]:
import sklearn as sk
import numpy as np

In [62]:
from sklearn import preprocessing as preproc

In [63]:
def preproc_df(df):
    for col in df.columns:
        if col in ["key", "loudness", "tempo"]:
            df[col] = df[col] = preproc.scale(df[col].values)

    df.dropna(inplace=True)
    data = []
    for row in df.values:
        data.append([np.array(row[:-1]),row[-1]])
        
    random.shuffle(data)
        
    X_data = []
    y_data = []
    
    for X,y in data:
        X_data.append(X)
        y_data.append(y-1)
    
        
    return np.array(X_data), np.array(y_data)
    

In [65]:
X,y = preproc_df(df)

In [66]:
# Making Model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(60, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(60, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(4, activation=tf.nn.softmax))

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [67]:
model.fit(X,y,epochs=10,validation_split=0.2)

Train on 3200 samples, validate on 800 samples
Epoch 1/10
3200/3200 [==============================] - 1s 255us/sample - loss: 0.8617 - accuracy: 0.6250 - val_loss: 0.5775 - val_accuracy: 0.7700
Epoch 2/10
3200/3200 [==============================] - 0s 37us/sample - loss: 0.5208 - accuracy: 0.7728 - val_loss: 0.5041 - val_accuracy: 0.7688
Epoch 3/10
3200/3200 [==============================] - 0s 37us/sample - loss: 0.4582 - accuracy: 0.8075 - val_loss: 0.4534 - val_accuracy: 0.8062
Epoch 4/10
3200/3200 [==============================] - 0s 41us/sample - loss: 0.4194 - accuracy: 0.8391 - val_loss: 0.4441 - val_accuracy: 0.8238
Epoch 5/10
3200/3200 [==============================] - 0s 40us/sample - loss: 0.3990 - accuracy: 0.8453 - val_loss: 0.4096 - val_accuracy: 0.8438
Epoch 6/10
3200/3200 [==============================] - 0s 42us/sample - loss: 0.3814 - accuracy: 0.8509 - val_loss: 0.4057 - val_accuracy: 0.8475
Epoch 7/10
3200/3200 [==============================] - 0s 39us/sample

In [68]:
# Save Model

MODEL_DIR = "models"
file_name = "simp_neural_net_2l_60n"

file_name = os.path.join(MODEL_DIR, file_name)
model.save(file_name)

INFO:tensorflow:Assets written to: models\simp_neural_net_2l_60n\assets


In [69]:
Song_categorizer_model = tf.keras.models.load_model(file_name)
predictions = Song_categorizer_model.predict(X)
X

array([[-8.93641985e-01,  1.01000000e-01,  9.58000000e-01, ...,
         3.16000000e-01,  7.66000000e-02,  9.95000000e-01],
       [-8.93641985e-01,  7.84000000e-02,  9.33000000e-01, ...,
         6.13000000e-01,  1.00000000e-01,  9.45000000e-01],
       [-6.16500728e-01,  1.57000000e-01,  9.38000000e-01, ...,
         5.35000000e-01,  3.12000000e-01,  7.97000000e-01],
       ...,
       [ 1.60062933e+00,  1.04000000e-01,  9.22000000e-05, ...,
         7.65000000e-01,  7.62000000e-01,  4.36000000e-01],
       [ 4.92064303e-01,  8.53000000e-02,  9.52000000e-01, ...,
         5.46000000e-01,  6.20000000e-02,  9.89000000e-01],
       [ 2.14923045e-01,  1.16000000e-01,  5.51000000e-01, ...,
         8.04000000e-01,  1.54000000e-01,  8.14000000e-01]])